# Кластеризация. Предобработка и обучение с частичным привлечением учителя


<hr>

С.Ю. Папулин (papulin.study@yandex.ru)

### Содержание

- [Предобработка данных для задачи классификации](#Предобработка-данных-для-задачи-классификации)
- [Обучение с частичным привлечением учителя](#Обучение-с-частичным-привлечением-учителя)
- [Источники](#Источники)

Подключение библиотек:

In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline

Функция загрузки исходного набора данных

In [ ]:
"""
Fashion MNIST Dataset: https://github.com/zalandoresearch/fashion-mnist
"""

from os import makedirs, remove
from os.path import exists, join
import gzip

from sklearn.datasets.base import RemoteFileMetadata, _fetch_remote
from sklearn.datasets import get_data_home
from sklearn.utils import Bunch

import numpy as np
import logging


logger = logging.getLogger(__name__)


ARCHIVES = [
    RemoteFileMetadata(
        filename='train-images-idx3-ubyte.gz',
        url='http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz',
        checksum=('3aede38d61863908ad78613f6a32ed271626dd12800ba2636569512369268a84')),
    RemoteFileMetadata(
        filename='train-labels-idx1-ubyte.gz',
        url='http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-labels-idx1-ubyte.gz',
        checksum=('a04f17134ac03560a47e3764e11b92fc97de4d1bfaf8ba1a3aa29af54cc90845')),
    RemoteFileMetadata(
        filename='test-images-idx3-ubyte.gz',
        url='http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-images-idx3-ubyte.gz',
        checksum=('346e55b948d973a97e58d2351dde16a484bd415d4595297633bb08f03db6a073')),
    RemoteFileMetadata(
        filename='test-labels-idx1-ubyte.gz',
        url='http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-labels-idx1-ubyte.gz',
        checksum=('67da17c76eaffca5446c3361aaab5c3cd6d1c2608764d35dfb1850b086bf8dd5'))
]


def fetch_fashion_mnist(data_home=None, download_if_missing=True, subset='all', return_X_y=False):  
    """
    Load the Fashion MNIST dataset (classification).
    
    Note: Based on https://github.com/scikit-learn/scikit-learn
    """
    data_home = get_data_home(data_home=data_home)
    if not exists(data_home):
        makedirs(data_home)

    for archive in ARCHIVES:
        filepath = join(data_home, archive.filename)
        if not exists(filepath):
            if not download_if_missing:
                raise IOError("Data not found and `download_if_missing` is False")
            logger.info('Downloading Fashion mnist from {} to {}'.format(
                archive.url, filepath))
            archive_path = _fetch_remote(archive, dirname=data_home)
        
    if return_X_y:
        
        DESCR = '''
            Fashion-MNIST is a dataset of Zalando's article images—consisting of 
            a training set of 60,000 examples and a test set of 10,000 examples. 
            Each example is a 28x28 grayscale image, associated with a label from 
            10 classes. We intend Fashion-MNIST to serve as a direct drop-in 
            replacement for the original MNIST dataset for benchmarking machine 
            learning algorithms. It shares the same image size and structure of 
            training and testing splits.
            '''
        
        feature_names = [
            'T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
            'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'
        ]
        
        
        if subset == 'train':
            X, y = _load_X_y(data_home, 'train')
            return Bunch(
                data=X,
                target=y,
                feature_names=feature_names,
                DESCR=DESCR
            )
        elif subset == 'test':
            X, y = _load_X_y(data_home, 'test')
            return Bunch(
                data=X,
                target=y,
                feature_names=feature_names,
                DESCR=DESCR
            )
        X_train, y_train = _load_X_y(data_home, 'train')
        X_test, y_test = _load_X_y(data_home, 'test')
        return Bunch(
            data={'train': X_train, 'test': X_test},
            target={'train': y_train, 'test': y_test},
            feature_names=feature_names,
            DESCR=DESCR
        )

        
def _load_X_y(path, subset='train'):
    """
    Load MNIST data from `path`.
    
    Note: Based on 
    https://github.com/zalandoresearch/fashion-mnist/blob/master/utils/mnist_reader.py
    """
    y_path = join(path, '{}-labels-idx1-ubyte.gz'.format(subset))
    X_path = join(path, '{}-images-idx3-ubyte.gz'.format(subset))

    with gzip.open(y_path, 'rb') as y_file:
        y = np.frombuffer(y_file.read(), dtype=np.uint8,
                               offset=8)
    with gzip.open(X_path, 'rb') as X_file:
        X = np.frombuffer(X_file.read(), dtype=np.uint8,
                               offset=16).reshape(len(y), 784)
    return X, y

## Предобработка данных для задачи классификации

Загрузка данных

In [ ]:
fashion_dataset = fetch_fashion_mnist(return_X_y=True)

print('Overview\n', fashion_dataset.DESCR)
print('Feature names\n', fashion_dataset.feature_names)

In [ ]:
IMAGE_INDX = 20

print('Image:')
plt.figure(figsize=[4, 4])
plt.imshow(fashion_dataset.data['train'][IMAGE_INDX].reshape(-1, 28))
plt.show()

print('Target:', fashion_dataset.target['train'][IMAGE_INDX])
print('Name:', fashion_dataset.feature_names[fashion_dataset.target['train'][IMAGE_INDX]])

Размерность данных

In [ ]:
fashion_dataset.data['train'].shape, fashion_dataset.data['test'].shape

In [ ]:
fashion_dataset.target['train'].shape, fashion_dataset.target['test'].shape

Обучающее и тестовое подмножества

In [ ]:
X_train_, X_test, y_train_, y_test = *fashion_dataset.data.values(), *fashion_dataset.target.values()

# Уменьшение количества элементов обучающего множества
X_train = X_train_[:10000]
y_train = y_train_[:10000]

# Уменьшение размера изображений
# X_train = X_train.reshape(-1, 28, 28)[:, ::2, ::2].reshape(-1, 14*14)

Обучения классификатора

In [ ]:
tick = time.time()
model = SVC(**{'C': 10, 'kernel': 'poly', 'gamma': 'scale'})
model.fit(X_train, y_train)
print("Time =", time.time() - tick)
model.score(X_test, y_test)

Формирование признаков посредством кластеризации

In [ ]:
tick = time.time()
cluster_model = KMeans(n_clusters=50, random_state=12345)
cluster_model.fit(X_train)
print("Time =", time.time() - tick)

In [ ]:
# Расстояние до центров кластеров
cluster_model.transform(X_train)[:1]

In [ ]:
cluster_model.transform(X_train).shape

In [ ]:
# Предсказание кластеров
print(np.argmin(cluster_model.transform(X_train), axis=1)[:5])
print(cluster_model.predict(X_train[:5]))

Классификатор с новым наборам признаков

In [ ]:
tick = time.time()
model = SVC(**{'C':10, 'kernel':'poly', 'gamma': 'scale'})
model.fit(cluster_model.transform(X_train), y_train)
print('Time =', time.time() - tick)
model.score(cluster_model.transform(X_test), y_test)

Реализация посредством `Pipeline`

In [ ]:
tick = time.time()
pipeline = Pipeline([
    ('cluster_model', KMeans(n_clusters=50, random_state=12345)),
    ('classifier', SVC(**{'C':10, 'kernel': 'poly', 'gamma': 'scale'})),
])
pipeline.fit(X_train, y_train)
print('Time =', time.time() - tick)
pipeline.score(X_test, y_test)

## Обучение с частичным привлечением учителя

In [ ]:
# Везьмем первые n наблюдений
N = 500
X_train_n = X_train[:N]
y_train_n = y_train[:N]

In [ ]:
# Обучаем модель классификации
tick = time.time()
model = SVC(**{'C': 10, 'kernel': 'poly', 'gamma': 'scale'})
# model = LogisticRegression()
model.fit(X_train_n, y_train_n)
print('Accuracy =', model.score(X_test, y_test))
print('Time =', time.time() - tick)

In [ ]:
# Обучаем модедь кластеризации
tick = time.time()
cluster_model = KMeans(n_clusters=N, random_state=12345)
cluster_model.fit(X_train)
print('Time =', time.time() - tick)

Разметка данных ближайших к кластерам

In [ ]:
# Индексы наблюдений с минимальным расстоянием до ближайщего кластера
indices = np.argmin(cluster_model.transform(X_train), axis=0)
indices

In [ ]:
cols = 10
row_num = -(-len(indices) // cols)

fig, axs = plt.subplots(row_num, cols, figsize=(14, 2*row_num), squeeze=False)
for i in range(row_num):
    for j in range(cols):
        indx = i * cols + j
        if indx >= len(indices):
            fig.delaxes(axs[i, j])
        else:
            image = X_train[indices[indx]].reshape(-1, 28)
            axs[i, j].imshow(image)
            axs[i, j].set_title(
                "cluster={}".format(indx))
            axs[i, j].axis("off")
# plt.tight_layout()
plt.show()

In [ ]:
# Массив наблюдений, соответствующих ранее полученным индексам
X_train_n_labeled = X_train[indices]

# Замечения: Эти значения должны быть внесены вручную на основе
# изображений выше. Однако здесь мы используем уже размеченный 
# набор с целевыми значениями
y_train_n_labeled = y_train[indices]

In [ ]:
# Обучение на новом наборе из N размеченных данных
model = SVC(**{'C': 10, 'kernel': 'poly', 'gamma': 'scale'})
model.fit(X_train_n_labeled, y_train_n_labeled)
print('Accuracy =', model.score(X_test, y_test))
print('Time =', time.time() - tick)

Разметка всего набора данных

In [ ]:
# Предсказание кластеров
с__pred = cluster_model.predict(X_train)

In [ ]:
# Сопоставим индексы кластеров и индексы классов (для размеченных вручную изображений)
y_train_labeled = y_train[indices[с__pred]]
y_train_labeled

In [ ]:
# Обучение на новых размеченных данных
tick = time.time()
model = SVC(**{'C': 10, 'kernel': 'poly', 'gamma': 'scale'})
model.fit(X_train, y_train_labeled)
print('Accuracy =', model.score(X_test, y_test))
print('Time =', time.time() - tick)

Приведенные выше способы подходят для повышения качества предсказания моделей при небольшом количестве размеченных данных. Если у нас достаточно большой набор размеченных данных, то не стоит ожидать значительного увеличения качества предсказания

##  Источники

Hands-on Machine Learning with Scikit-Learn, Keras, and TensorFlow by Aurélien Géron